In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [2]:
#setup
df = pd.read_csv("consumption.csv")
weather_avg = pd.read_csv('weather-avg.csv')
weather_min = pd.read_csv('weather-min.csv')
weather_max = pd.read_csv('weather-max.csv')
info = pd.read_csv('addinfo.csv')
weather_avg = weather_avg.set_index("meter_id")
weather_min = weather_min.set_index("meter_id")
weather_max = weather_max.set_index("meter_id")
info = info.set_index("meter_id")
weather_avg_sorted = weather_avg.reindex(df.iloc[:,0])
weather_min_sorted = weather_min.reindex(df.iloc[:,0])
weather_max_sorted = weather_max.reindex(df.iloc[:,0])
info_sorted = info.reindex(df.iloc[:,0])
brinfo=info_sorted['num_bedrooms']
values = {'num_bedrooms' : brinfo.mean()}
info_filled_br = info_sorted.fillna(value = values)
brinfo_filled = info_filled_br['num_bedrooms']
#functions
def get_monthi(n):
    begin=48*31*(n-1)+1
    end=48*31*n
    if n==1:
        begin=1
    if n>1:
        end-=3*48
    if n>2:
        begin-=3*48
    if n>3:
        end-=48
    if n>4:
        begin-=48
    if n>5:
        end-=48
    if n>6:
        begin-=48
    if n>8:
        end-=48
    if n>9:
        begin-=48
    if n>10:
        end-=48
    if n>11:
        begin-=48
    return begin,end

def get_mean_temp(row,month):
    """
    row: is the row (meter_id) we would like to get the average temperature for.
    month: which month (columns) we would get the average temperature for.
    returns: the average temperature for a specific meter_id for a specific month.
    """
    if month==1:
        return row.loc[:,"2017-01-01 00:00:00":"2017-01-31 00:00:00"].mean(1)
    elif month==2:
        return row.loc[:,"2017-02-01 00:00:00":"2017-02-28 00:00:00"].mean(1)
    elif month==3:
        return row.loc[:,"2017-03-01 00:00:00":"2017-03-31 00:00:00"].mean(1)
    elif month==4:
        return row.loc[:,"2017-04-01 00:00:00":"2017-04-30 00:00:00"].mean(1)
    elif month==5:
        return row.loc[:,"2017-05-01 00:00:00":"2017-05-31 00:00:00"].mean(1)
    elif month==6:
        return row.loc[:,"2017-06-01 00:00:00":"2017-06-30 00:00:00"].mean(1)
    elif month==7:
        return row.loc[:,"2017-07-01 00:00:00":"2017-07-31 00:00:00"].mean(1)
    elif month==8:
        return row.loc[:,"2017-08-01 00:00:00":"2017-08-31 00:00:00"].mean(1)
    elif month==9:
        return row.loc[:,"2017-09-01 00:00:00":"2017-09-30 00:00:00"].mean(1)
    elif month==10:
        return row.loc[:,"2017-10-01 00:00:00":"2017-10-31 00:00:00"].mean(1)
    elif month==11:
        return row.loc[:,"2017-11-01 00:00:00":"2017-11-30 00:00:00"].mean(1)
    elif month==12:
        return row.loc[:,"2017-12-01 00:00:00":"2017-12-31 00:00:00"].mean(1)
    else:
        print("Error: this is not a valid input for month")

In [151]:
NaN_t=1200
En_con=[]
l_En_con=[]

for i in range(df.shape[0]): # loop over all users
    meter=df.iloc[i] # = row number i  
    
    for m in range (2,13): # loop over all months
        #load current month
        bi, ei =get_monthi(m)               #bi (begin index) ei(end index)
        month=meter[bi:ei]  #data for the month m for the row user (row) i
        #Load last month
        lbi,lei=get_monthi(m-1)             #lbi (last begin index) lei (last end index)
        last_month=meter[lbi:lei] 
        
        # Count the amount of NaN's in current and last month      
        n_NaN=month.isnull().sum() 
        l_n_NaN=last_month.isnull().sum()
        
        # Check if there are any NaN's in current and last month
        if n_NaN<NaN_t and l_n_NaN==0:
            En_con.append(month.mean())
            # number of days in the last month
            n_day=np.int((lei-lbi+1)/48)
            # daily consumption last month
            # all months will be considered 31 days since the model needs a constant input size
            # months with less then 31 days will get the averagen comsumption assigned to the remaining days
            daily_con=np.zeros(31)
            if n_day < 31:
                #fill in daily data with real data
                for j in range(n_day):
                    daily_con[j]=last_month[j*48:(j+1)*48].mean()
                #fill in daily data with the mean
                mean_month=last_month.mean()
                for j in range(n_day,31):
                    daily_con[j]=mean_month
            else:
                for j in range(31):
                    daily_con[j]=last_month[j*48:(j+1)*48].mean()
            if np.isnan(np.sum(daily_con)):
                print("m=%i ,index=%i" %(m-1,i))
                print(last_month)
                print(daily_con)
            l_En_con.append(daily_con)  
                
            

In [18]:
#number of days in a month
b,e=get_monthi(12)
(e-b+1)/48

31.0

In [77]:
#some tests
meter=df.iloc[0] # = row number i  
l_begin_index,l_end_index=get_monthi(12)
last_month=meter[l_begin_index:l_end_index+1] #load last 
meter[l_begin_index]
nul=np.zeros(31)
nul[2]=3
i=30
#print(last_month[i*48:(i+1)*48])
for i in range(30,31): print(i)

30


In [124]:
#print(l_En_con)
npl_En_con=np.array(l_En_con)
npEn_con = np.zeros(len(En_con))
npEn_con=np.array(En_con).ravel()
print(npl_En_con.shape) #only 3588 data points/months remain

(3588, 31)


In [125]:
X_train = npl_En_con[:-1000] #70% trainning set

X_test = npl_En_con[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)

(2588, 31)


In [127]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.8f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f'
      % r2_score(y_test, y_pred))

Coefficients: 
 [[ 0.01651568  0.00458183  0.02064142  0.06776875 -0.02990141 -0.02470747
   0.04329187  0.03475493  0.02203448  0.03075617  0.00149132  0.01580567
  -0.01895977  0.0016443  -0.00715614  0.01430695 -0.00110777  0.00722975
   0.03515705  0.06505902  0.02848955  0.09322874  0.00034209  0.06460545
   0.00199705  0.03252616  0.03168476  0.01965189  0.01201485  0.09522453
   0.19844906]]
Mean squared error: 0.00165948
Coefficient of determination: 0.8958


The last coefficient seems to be the most important. This is a result of adding the imaginary extra days on 6 of 12 months. I'm not sure if the other coefficients actually contributed or that they're just noise. There is a significant improvement to the r^2 value but this might have something to do with the seleceted data and test set aswell. To check this we'll have to run an extra check we're we only use the mean as input over this data set just like in "linear_model_past_data"

We'll now construct a array with all the means of the previously used months. It would be easier to loop over the previously generater l_En_con array but this would result in a small error because of the imputed data at the end of some months

In [135]:
l_En_con_mean=[]
for i in range(df.shape[0]): # loop over all users
    meter=df.iloc[i] # = row number i  
    
    for m in range (2,13): # loop over all months
        #load current month
        bi, ei =get_monthi(m)               #bi (begin index) ei(end index)
        month=meter[bi:ei]  #data for the month m for the row user (row) i
        #Load last month
        lbi,lei=get_monthi(m-1)             #lbi (last begin index) lei (last end index)
        last_month=meter[lbi:lei] 
        
        # Count the amount of NaN's in current and last month      
        n_NaN=month.isnull().sum() 
        l_n_NaN=last_month.isnull().sum()
        
        # Check if there are any NaN's in current and last month
        if n_NaN<NaN_t and l_n_NaN==0:
            l_En_con_mean.append(last_month.mean())

In [143]:
npl_En_con_mean = np.zeros(len(l_En_con_mean))
npl_En_con_mean=np.array(l_En_con_mean).ravel()
npl_En_con_mean=npl_En_con_mean.reshape(-1,1)
print(npl_En_con_mean.shape)
print(npl_En_con_mean)

(3588, 1)
[[0.56780959]
 [0.13408617]
 [0.11767755]
 ...
 [0.49908637]
 [0.24658652]
 [0.13918485]]


Now we make a new linear model

In [144]:
X_train = npl_En_con_mean[:-1000] #70% trainning set

X_test = npl_En_con_mean[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.8f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f'
      % r2_score(y_test, y_pred))

Coefficients: 
 [[0.85518305]]
Mean squared error: 0.00183649
Coefficient of determination: 0.8846


It seems like most of the improments were due to the smaller data set and not the inclusiong of the daily information. It could also be that our linear model can not take advantages of the extra information and just makes a sligtly different or more randomised mean of our months

Now we'll check what the effect is of not imputing any data an just trowing away the end of the months

In [146]:
NaN_t=1200
En_con=[]
l_En_con_short=[]

for i in range(df.shape[0]): # loop over all users
    meter=df.iloc[i] # = row number i  
    
    for m in range (2,13): # loop over all months
        #load current month
        bi, ei =get_monthi(m)               #bi (begin index) ei(end index)
        month=meter[bi:ei]  #data for the month m for the row user (row) i
        #Load last month
        lbi,lei=get_monthi(m-1)             #lbi (last begin index) lei (last end index)
        last_month=meter[lbi:lei] 
        
        # Count the amount of NaN's in current and last month      
        n_NaN=month.isnull().sum() 
        l_n_NaN=last_month.isnull().sum()
        
        # Check if there are any NaN's in current and last month
        if n_NaN<NaN_t and l_n_NaN==0:
            En_con.append(month.mean())
            # number of days in the last month
            n_day=np.int((lei-lbi+1)/48)
            # daily consumption last month
            # all months will be considered 31 days since the model needs a constant input size
            # months with less then 31 days will get the averagen comsumption assigned to the remaining days
            daily_con=np.zeros(28)
            for j in range(28):
                daily_con[j]=last_month[j*48:(j+1)*48].mean()
            l_En_con_short.append(daily_con)  

In [147]:
npl_En_con_short=np.array(l_En_con_short)
npEn_con = np.zeros(len(En_con))
npEn_con=np.array(En_con).ravel()
print(npl_En_con_short.shape) #only 3588 data points/months remain

(3588, 28)


Training the linear model again

In [148]:
X_train = npl_En_con_short[:-1000] #70% trainning set

X_test = npl_En_con_short[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.8f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f'
      % r2_score(y_test, y_pred))

Coefficients: 
 [[ 0.00552543  0.00902142  0.01006986  0.08388736 -0.03056483 -0.03082037
   0.04670765  0.0474355   0.05305962  0.04668855  0.01867475  0.01445524
  -0.00073413  0.00802702  0.01864846  0.03753931  0.00350489  0.00993871
   0.04227988  0.07914947  0.02177436  0.09020664  0.0189688   0.0740619
   0.01166754  0.04242231  0.07178077  0.05994506]]
Mean squared error: 0.00172722
Coefficient of determination: 0.8915


The results are still better even without the added data

### Some polynomail tests

months with imputed data

In [159]:
#Polynomial regression:
X_train = npl_En_con[:-1000] #70% trainning set

X_test = npl_En_con[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)

def polfit(X_train,X_test,y_train,y_test,deg,printcoef=False):
    # Fitting Polynomial Regression to the dataset
    # source: https://towardsdatascience.com/machine-learning-polynomial-regression-with-python-5328e4e8a386
    # Create polynomial regression object
    polynom_degree = deg

    poly_reg = PolynomialFeatures(degree = polynom_degree)

    X_poly = poly_reg.fit_transform(X_train) # Fit to data, then transform it to 
    #a new feature matrix consisting ofall polynomial combinations of the features

    pol_reg = LinearRegression()

    # Train the model using the training sets
    pol_reg.fit(X_poly, y_train)

    # polyfitting the training data:
    poly_vals = pol_reg.predict(poly_reg.fit_transform(X_train))

    # Make predictions using the testing set
    y_pred_poly = pol_reg.predict(poly_reg.fit_transform(X_test))

    # The coefficients
    if printcoef:
        print('Coefficients: \n', pol_reg.coef_)
        
    # The mean squared error
    print('Mean squared error: %.8f'
      % mean_squared_error(y_test, y_pred_poly))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.8f'
      % r2_score(y_test, y_pred_poly))


In [160]:
#imputed data
X_train = npl_En_con[:-1000] #70% trainning set

X_test = npl_En_con[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)

polfit(X_train,X_test,y_train,y_test,2)

#mean vals
X_train = npl_En_con_mean[:-1000] #70% trainning set

X_test = npl_En_con_mean[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)

polfit(X_train,X_test,y_train,y_test,2)

#drop ens of months

X_train = npl_En_con_short[:-1000] #70% trainning set

X_test = npl_En_con_short[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)

polfit(X_train,X_test,y_train,y_test,2)

Mean squared error: 0.00369047
Coefficient of determination: 0.76817884
Mean squared error: 0.00183721
Coefficient of determination: 0.88459352
Mean squared error: 0.00306837
Coefficient of determination: 0.80725664


Maybe a neural netword can use this extra data

In [161]:
import tensorflow as tf

In [173]:
#imputed data
X_train = npl_En_con[:-1000] #70% trainning set

X_test = npl_En_con[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)

In [174]:
# construct the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, input_dim=31, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1)
])
#print(model.summary())
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[tf.keras.metrics.MeanSquaredError()])
print("Start training")
# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

# evaluate the model
mse = model.evaluate(X_test, y_test)[1]
print(f'MSE: {mse}')
y_pred_comb=model.predict(X_test)
print('Mean squared error: %.8f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.8f'
      % r2_score(y_test, y_pred))

Start training
Train on 2588 samples, validate on 1000 samples
Epoch 1/50
2588/2588 [==============================] - 1s 331us/sample - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 2/50
2588/2588 [==============================] - 0s 88us/sample - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0019 - val_mean_squared_error: 0.0019
Epoch 3/50
2588/2588 [==============================] - 0s 101us/sample - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0025 - val_mean_squared_error: 0.0025
Epoch 4/50
2588/2588 [==============================] - 0s 90us/sample - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 5/50
2588/2588 [==============================] - 0s 109us/sample - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 6/50
2588/2588 [==============================] - 0s 94us/sample - loss: 0.0011 - mean_square

Epoch 48/50
2588/2588 [==============================] - 0s 94us/sample - loss: 8.0338e-04 - mean_squared_error: 8.0338e-04 - val_loss: 0.0018 - val_mean_squared_error: 0.0018
Epoch 49/50
2588/2588 [==============================] - 0s 89us/sample - loss: 8.6574e-04 - mean_squared_error: 8.6574e-04 - val_loss: 0.0016 - val_mean_squared_error: 0.0016
Epoch 50/50
1000/1000 [==============================] - ETA: 0s - loss: 0.0014 - mean_squared_error: 0.00 - 0s 39us/sample - loss: 0.0018 - mean_squared_error: 0.0018
MSE: 0.001844163634814322
Mean squared error: 0.00172722
Coefficient of determination: 0.89150260


In [166]:
#shortened data
X_train = npl_En_con_short[:-1000] #70% trainning set

X_test = npl_En_con_short[-1000:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-1000]
y_train = npEn_con[:-1000].reshape(-1,1)

y_test = npEn_con[-1000:]
y_test = y_test.reshape(-1,1)

In [171]:
# construct the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_dim=28, activation='relu'),
    tf.keras.layers.Dense(1)
])
#print(model.summary())
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[tf.keras.metrics.MeanSquaredError()])
print("Start training")
# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

# evaluate the model
mse = model.evaluate(X_test, y_test)[1]
print(f'MSE: {mse}')
y_pred_comb=model.predict(X_test)
print('Mean squared error: %.8f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.8f'
      % r2_score(y_test, y_pred))

Start training
Train on 2588 samples, validate on 1000 samples
Epoch 1/50
2588/2588 [==============================] - 1s 247us/sample - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0042 - val_mean_squared_error: 0.0042
Epoch 2/50
2588/2588 [==============================] - 0s 129us/sample - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.0038 - val_mean_squared_error: 0.0038
Epoch 3/50
2588/2588 [==============================] - 0s 60us/sample - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0032 - val_mean_squared_error: 0.0032
Epoch 4/50
2588/2588 [==============================] - 0s 63us/sample - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0030 - val_mean_squared_error: 0.0030
Epoch 5/50
2588/2588 [==============================] - 0s 70us/sample - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0031 - val_mean_squared_error: 0.0031
Epoch 6/50
2588/2588 [==============================] - 0s 49us/sample - loss: 0.0017 - mean_squared

Epoch 49/50
2588/2588 [==============================] - 0s 49us/sample - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 50/50
1000/1000 [==============================] - 0s 24us/sample - loss: 0.0017 - mean_squared_error: 0.0017
MSE: 0.001739345956593752
Mean squared error: 0.00172722
Coefficient of determination: 0.89150260


We'll now allow months with NaN data to see if this can improve our model

In [175]:
NaN_t=1200
NaN_t2=700
En_con=[]
l_En_con_short=[]

for i in range(df.shape[0]): # loop over all users
    meter=df.iloc[i] # = row number i  
    
    for m in range (2,13): # loop over all months
        #load current month
        bi, ei =get_monthi(m)               #bi (begin index) ei(end index)
        month=meter[bi:ei]  #data for the month m for the row user (row) i
        #Load last month
        lbi,lei=get_monthi(m-1)             #lbi (last begin index) lei (last end index)
        last_month=meter[lbi:lei] 
        
        # Count the amount of NaN's in current and last month      
        n_NaN=month.isnull().sum() 
        l_n_NaN=last_month.isnull().sum()
        
        # Check if there are any NaN's in current and last month
        if n_NaN<NaN_t and l_n_NaN<NaN_t2:
            En_con.append(month.mean())
            # number of days in the last month
            n_day=np.int((lei-lbi+1)/48)
            # daily consumption last month
            daily_con=np.zeros(28)
            mean_lm=last_month.mean()
            for j in range(28):
                daily_con[j]=last_month[j*48:(j+1)*48].mean()
                #if a day happens to be full of NaN we will impute this by the mean
                if np.isnan(daily_con[j]):
                    daily_con[j]=mean_lm
            l_En_con_short.append(daily_con) 

In [176]:
npl_En_con_short=np.array(l_En_con_short)
print(npl_En_con_short.shape) 

(17756, 28)


In [179]:
splitnum=np.int(0.3*npl_En_con_short.shape[0])
print(splitnum)

5326


In [180]:
splitnum=np.int(0.3*npl_En_con_short.shape[0])
X_train = npl_En_con_short[:-splitnum] #70% trainning set

X_test = npl_En_con_short[-splitnum:] #  last 6322 elements are the test set


# Split the targets into training/testing sets
y_train = npEn_con[:-splitnum]
y_train = npEn_con[:-splitnum].reshape(-1,1)

y_test = npEn_con[-splitnum:]
y_test = y_test.reshape(-1,1)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.8f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f'
      % r2_score(y_test, y_pred))

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.